In [24]:
import math
import random
from collections import deque
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim

# ----------------------------
# Environment Simulator
# ----------------------------
class EnvironmentSimulator:
    def __init__(self):
        self.map = [
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
            [1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
            [1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
            [1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1],
            [1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,1],
            [1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1],
            [1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1],
            [1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1],
            [1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1],
            [1,0,0,0,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1],
            [1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1],
            [1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1],
            [1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
        ]
        self.player_gun = random.randint(0,1)
        self.player_pos = self.initialize_position()
        self.player_orientation = random.randint(-180,180)
        self.player_health = 10

        self.opponent_gun = random.randint(0,1)
        self.opponent_pos = self.initialize_position()
        self.opponent_orientation = random.randint(-180,180)
        self.opponent_health = 10

    def initialize_position(self):
        pos = (random.randint(1,13), random.randint(1,18))
        while self.map[pos[0]][pos[1]] == 1:
            pos = (random.randint(1,13), random.randint(1,18))
        return [pos[0], pos[1]]

    def move(self, offset, movePlayer=True):
        if movePlayer:
            nr = self.player_pos[0] + offset[0]
            nc = self.player_pos[1] + offset[1]
            if self.map[nr][nc] != 1:
                self.player_pos = [nr,nc]
        else:
            nr = self.opponent_pos[0] + offset[0]
            nc = self.opponent_pos[1] + offset[1]
            if self.map[nr][nc] != 1:
                self.opponent_pos = [nr,nc]

    def turn(self, turnLeft=True, turnPlayer=True):
        if turnPlayer:
            self.player_orientation += 5 if turnLeft else -5
            self.player_orientation = self.mapToRange(self.player_orientation)
        else:
            self.opponent_orientation += 5 if turnLeft else -5
            self.opponent_orientation = self.mapToRange(self.opponent_orientation)

    def mapToRange(self, deg):
        deg %= 360
        if deg > 180: deg -= 360
        return deg

    def switchWeapon(self, switchPlayerWeapon=True):
        if switchPlayerWeapon:
            self.player_gun = not self.player_gun
        else:
            self.opponent_gun = not self.opponent_gun

    def shoot(self, playerShooting=True):
        if playerShooting:
            ang = math.degrees(math.atan2(
                self.opponent_pos[0]-self.player_pos[0],
                self.opponent_pos[1]-self.player_pos[1]))
            diff = ang - self.player_orientation
            dist = math.hypot(
                self.opponent_pos[0]-self.player_pos[0],
                self.opponent_pos[1]-self.player_pos[1])
            if self.player_gun == 1:
                if abs(diff)<10 and not self.has_obstacle_between() and dist*32<=300:
                    self.opponent_health -= 1
                    return True
            else:
                if abs(diff)<10 and not self.has_obstacle_between() and dist*32<=100:
                    self.opponent_health -= 3
                    return True
        else:
            ang = math.degrees(math.atan2(
                self.player_pos[0]-self.opponent_pos[0],
                self.player_pos[1]-self.opponent_pos[1]))
            diff = ang - self.opponent_orientation
            dist = math.hypot(
                self.player_pos[0]-self.opponent_pos[0],
                self.player_pos[1]-self.opponent_pos[1])
            if self.opponent_gun == 1:
                if abs(diff)<10 and not self.has_obstacle_between() and dist*32<=300:
                    self.player_health -= 1
                    return True
            else:
                if abs(diff)<10 and not self.has_obstacle_between() and dist*32<=100:
                    self.player_health -= 3
                    return True

    def perform(self, action, movePlayer=True):
        # perform on player or opponent
        if action == 0:
            self.shoot(movePlayer)
        elif action == 1:
            self.move((-1, 0), movePlayer)
        elif action == 2:
            self.move((1, 0), movePlayer)
        elif action == 3:
            self.move((0, -1), movePlayer)
        elif action == 4:
            self.move((0, 1), movePlayer)
        elif action == 5:
            self.move((-1, -1), movePlayer)
        elif action == 6:
            self.move((-1, 1), movePlayer)
        elif action == 7:
            self.move((1, -1), movePlayer)
        elif action == 8:
            self.move((1, 1), movePlayer)
        elif action == 9:
            self.turn(True, movePlayer)
        elif action == 10:
            self.turn(False, movePlayer)
        elif action == 11:
            self.switchWeapon(movePlayer)

    def ray_distance(self, angle_deg, max_dist=None):
        if max_dist is None:
            max_dist = max(len(self.map), len(self.map[0]))
        rad = math.radians(angle_deg)
        dx, dy = math.cos(rad), math.sin(rad)
        px, py = self.player_pos[1], self.player_pos[0]
        for d in range(1, max_dist+1):
            x = int(round(px + dx*d))
            y = int(round(py + dy*d))
            if not (0 <= y < len(self.map) and 0 <= x < len(self.map[0])):
                return d
            if self.map[y][x] == 1:
                return d
        return max_dist

    def bresenham_line(self, x0, y0, x1, y1):
        pts = []
        dx = abs(x1-x0); dy = -abs(y1-y0)
        sx = 1 if x0<x1 else -1; sy = 1 if y0<y1 else -1
        err = dx+dy
        while True:
            pts.append((x0,y0))
            if x0==x1 and y0==y1: break
            e2 = 2*err
            if e2 >= dy: err += dy; x0 += sx
            if e2 <= dx: err += dx; y0 += sy
        return pts

    def has_obstacle_between(self):
        line = self.bresenham_line(
            self.player_pos[1], self.player_pos[0],
            self.opponent_pos[1], self.opponent_pos[0])
        for x,y in line[1:-1]:
            if self.map[y][x] == 1: return True
        return False

    def is_terminal(self):
        return self.player_health == 0

    def reset(self):
        self.player_gun = random.randint(0,1)
        self.player_pos = self.initialize_position()
        self.player_orientation = random.randint(-180,180)
        self.player_health = 10
        self.opponent_gun = random.randint(0,1)
        self.opponent_pos = self.initialize_position()
        self.opponent_orientation = random.randint(-180,180)
        self.opponent_health = 10

# ----------------------------
# Q-Network & Replay Buffer
# ----------------------------
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=256):
        super().__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    def add(self, s,a,r,ns,d):
        self.buffer.append((s,a,r,ns,d))
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        s,a,r,ns,d = map(np.array, zip(*batch))
        return (torch.FloatTensor(s),
                torch.LongTensor(a),
                torch.FloatTensor(r),
                torch.FloatTensor(ns),
                torch.FloatTensor(d))
    def __len__(self):
        return len(self.buffer)

# ----------------------------
# DQN Agent with Metrics
# ----------------------------
class DQNAgent:
    def __init__(self, state_size, action_size,
                 lr=1e-3, gamma=0.99,
                 epsilon_start=1.0, epsilon_end=0.01, epsilon_decay=0.995,
                 target_update_freq=10, tau=None, opponent=False):
        self.state_size    = state_size
        self.action_size   = action_size
        self.q_network     = QNetwork(state_size, action_size)
        self.target_network= QNetwork(state_size, action_size)
        self.optimizer     = optim.Adam(self.q_network.parameters(), lr=lr)
        self.replay_buffer = ReplayBuffer(10000)
        self.batch_size    = 64
        self.gamma         = gamma
        self.environment_simulator = EnvironmentSimulator()
        self.player        = not opponent
        self.prev_action = None
        self.prev_position = None
        self.prev_dist = None          # 上一步與敵人的距離
        self.same_action_run = 0       # 連續同動作次數


        # epsilon-greedy
        self.epsilon       = epsilon_start
        self.epsilon_end   = epsilon_end
        self.epsilon_decay = epsilon_decay

        # target update
        self.target_update_freq = target_update_freq
        self.tau           = tau
        self._train_steps  = 0

        # Metrics
        self.hit_count        = 0
        self.miss_count       = 0
        self.kill_count       = 0
        self.move_count       = 0
        self.turn_left_count  = 0
        self.turn_right_count = 0
        self.switch_count     = 0

        self.update_target()

    def reset_metrics(self):
        self.hit_count        = 0
        self.miss_count       = 0
        self.kill_count       = 0
        self.move_count       = 0
        self.turn_left_count  = 0
        self.turn_right_count = 0
        self.switch_count     = 0
        self.prev_action = None
        self.prev_position = None
        self.prev_dist = None
        self.same_action_run = 0



    def update_target(self):
        self.target_network.load_state_dict(self.q_network.state_dict())

    def select_action(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, self.action_size-1)
        with torch.no_grad():
            s = torch.FloatTensor(state).unsqueeze(0)
            return self.q_network(s).argmax().item()

    def train_step(self):
        if len(self.replay_buffer) < self.batch_size: return
        s,a,r,ns,d = self.replay_buffer.sample(self.batch_size)
        q_vals = self.q_network(s).gather(1, a.unsqueeze(1)).squeeze(1)
        with torch.no_grad():
            max_next = self.target_network(ns).max(1)[0]
            target_q = r + (1-d)*self.gamma*max_next
        loss = nn.MSELoss()(q_vals, target_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # epsilon decay
        self.epsilon = max(self.epsilon*self.epsilon_decay, self.epsilon_end)
        # target update
        if self.tau is None:
            self._train_steps += 1
            if self._train_steps % self.target_update_freq == 0:
                self.update_target()
        else:
            for tp,p in zip(self.target_network.parameters(),
                            self.q_network.parameters()):
                tp.data.copy_(self.tau*p.data + (1-self.tau)*tp.data)

    def step(self):
        state  = self.getState()
        action = self.select_action(state)

        # record HP before shoot
        prev_hp = (self.environment_simulator.opponent_health
                   if self.player else
                   self.environment_simulator.player_health)

        # perform with correct actor
        self.environment_simulator.perform(action, movePlayer=self.player)
        next_state = self.getState()

        # record HP after
        new_hp = (self.environment_simulator.opponent_health
                  if self.player else
                  self.environment_simulator.player_health)

        # metrics: shoot
        if action == 0:
            if new_hp < prev_hp:
                self.hit_count += 1
                if new_hp <= 0:
                    self.kill_count += 1
            else:
                self.miss_count += 1
        # metrics: other
        if action in range(1,9):
            self.move_count += 1
        elif action == 9:
            self.turn_left_count += 1
        elif action == 10:
            self.turn_right_count += 1
        elif action == 11:
            self.switch_count += 1

        reward = self.calculate_reward(state, action, next_state)
        done   = self.environment_simulator.is_terminal()
        self.replay_buffer.add(state, action, reward, next_state, done)
        self.train_step()
        # 更新前一動作與位置
        # 更新歷史
        env = self.environment_simulator
        self.prev_action  = action
        self.prev_position = env.player_pos[:] if self.player else env.opponent_pos[:]
        # 以 tile 為單位距離
        self.prev_dist = math.hypot(next_state[2] * len(env.map),
                                    next_state[3] * len(env.map[0]))
        # 連續同動作統計
        if self.same_action_run and action == self.prev_action:
            self.same_action_run += 1
        else:
            self.same_action_run = 1


        return next_state, reward, done

    def calculate_reward(self, state, action, next_state):
        """
        ──────────────────────────────────────────────────────────────
        state[2],state[3]  : 敵人相對 row/col
        state[6]           : 1=步槍(遠程) 0=散彈(近程)
        state[8]           : 自身朝向 0~1
        ray_distance 前/左30/右30 已在 getState() 給出 (state[8]~[10])
        ──────────────────────────────────────────────────────────────
        """
        env = self.environment_simulator
        H, W = len(env.map), len(env.map[0])

        # === 幾何量 ===
        dist_prev = math.hypot(state[2]*H,  state[3]*W)
        dist_next = math.hypot(next_state[2]*H, next_state[3]*W)

        ori_deg = (env.player_orientation % 360)
        ang_to_enemy = self.angle_between_player_and_opponent((state[2], state[3]))
        ang_diff_prev = abs(self.angleDiff(ang_to_enemy, ori_deg))

        # 下一步角度誤差
        ang_to_enemy_next = self.angle_between_player_and_opponent((next_state[2], next_state[3]))
        ang_diff_next = abs(self.angleDiff(ang_to_enemy_next, ori_deg))

        # 可視 + 遮蔽
        in_fov = ang_diff_prev <= 30
        visible = in_fov and not env.has_obstacle_between()

        # === 基礎命中判定 ===
        hit  = (next_state[4] < state[4])
        kill = (next_state[4] <= 0)

        reward = 0.0

        # ───────────────── 射擊動作 (0) ─────────────────
        if action == 0:
            if visible and hit:
                reward += 200.0
                if kill:
                    reward += 1000.0
                # combo: 前一步是 turn 而這一步命中
                if self.prev_action in (9, 10):
                    reward += 50.0
            elif visible and not hit:
                reward -= 5.0
            else:
                reward -= 2.0

        # ───────────────── 移動動作 (1~8) ───────────────
        elif action in range(1, 9):
            # 有效移動
            curr_pos = env.player_pos if self.player else env.opponent_pos
            if self.prev_position and curr_pos != self.prev_position:
                reward += 1.0
            else:
                reward -= 0.5

            # 若敵人可見且更接近
            if visible and dist_next < dist_prev:
                reward += 0.3

            # combo: move → turn (若角度改善)
            if self.prev_action in range(1, 9) and ang_diff_next < ang_diff_prev:
                reward += 0.3

        # ───────────────── 轉向動作 (9,10) ──────────────
        elif action in (9, 10):
            if not visible:
                reward += 0.3      # 探索視野
            else:
                # 若角度誤差縮小
                if ang_diff_next < ang_diff_prev:
                    reward += 0.5
                else:
                    reward -= 0.2

        # ───────────────── 武器切換 (11) ───────────────
        elif action == 11:
            close = dist_prev < 4.0
            use_rifle = bool(state[6])  # 1=rifle,0=shotgun
            correct = (close and not use_rifle) or (not close and use_rifle)
            if correct:
                reward += 2.0
                # combo: move → switchWeapon 當距離剛跨閾值
                if self.prev_action in range(1, 9):
                    reward += 1.0
            else:
                reward -= 0.5

        # ───────────────── 動作多樣性 & 連續懲罰 ───────
        if self.prev_action is not None and action != self.prev_action:
            reward += 3
        if self.same_action_run >= 3:          # 連續4步同動作
            reward -= 5

        return reward



    def getState(self):
        env = self.environment_simulator
        H, W = len(env.map), len(env.map[0])
        pr, pc = env.player_pos
        row_norm, col_norm = pr/H, pc/W
        or_, oc = env.opponent_pos
        d_row, d_col = (or_-pr)/H, (oc-pc)/W
        opp_hp, ply_hp = env.opponent_health/10.0, env.player_health/10.0
        gun = float(env.player_gun)
        ori = (env.player_orientation % 360)/360.0
        def ray(a): return env.ray_distance(a, 10)/10.0
        front  = ray(ori*360)
        left30 = ray((ori*360+30)%360)
        right30= ray((ori*360-30)%360)
        return [row_norm, col_norm, d_row, d_col, opp_hp, ply_hp, gun, ori, front, left30, right30]

    def magnitude(self, v):
        return math.hypot(v[0], v[1])

    def mapToRange(self, deg):
        deg %= 360
        if deg>180: deg -= 360
        return deg

    def angle_between_player_and_opponent(self, rel):
        return self.mapToRange(math.degrees(math.atan2(rel[0], rel[1])))

    def angleDiff(self, a1, a2):
        d = a1 - a2
        if d>180: d -= 360
        if d<-180: d += 360
        return d

    def save_model(self, filename="dqn_model.pth"):
        os.makedirs("models1", exist_ok=True)
        torch.save(self.q_network.state_dict(), os.path.join("models1", filename))

    def load_model(self, filename="dqn_model.pth"):
        self.q_network.load_state_dict(torch.load(os.path.join("models1", filename)))
        self.q_network.eval()

# ----------------------------
# Training Loop
# ----------------------------
NUMBER_OF_EPISODES = 2000
NUMBER_OF_STEPS    = 2500

player_agent   = DQNAgent(11, 12)
opponent_agent = DQNAgent(11, 12, opponent=True)

def reset():
    player_agent.environment_simulator.reset()
    es = player_agent.environment_simulator
    osim = opponent_agent.environment_simulator
    osim.player_pos         = es.opponent_pos[:]
    osim.player_health      = es.opponent_health
    osim.player_orientation = es.opponent_orientation
    osim.player_gun         = es.opponent_gun
    osim.opponent_pos       = es.player_pos[:]
    osim.opponent_health    = es.player_health
    osim.opponent_orientation = es.player_orientation
    osim.opponent_gun       = es.player_gun

pretrained_path = "models/player_winner.pth"
if os.path.exists(pretrained_path):
    print("[INFO] Loading pretrained weights...")
    player_agent.load_model("winner.pth")
    opponent_agent.load_model("winner.pth")

for ep in range(NUMBER_OF_EPISODES):
    player_agent.reset_metrics()
    opponent_agent.reset_metrics()
    p_total, o_total = 0.0, 0.0
    p_done, o_done = False, False

    for step in range(NUMBER_OF_STEPS):
        p_ns, p_r, p_done = player_agent.step()
        o_ns, o_r, o_done = opponent_agent.step()

        p_total += p_r
        o_total += o_r

        if p_done or o_done:
            break

        # 同步環境
        es = player_agent.environment_simulator
        osim = opponent_agent.environment_simulator
        osim.player_pos         = es.opponent_pos[:]
        osim.player_health      = es.opponent_health
        osim.player_orientation = es.opponent_orientation
        osim.player_gun         = es.opponent_gun
        osim.opponent_pos       = es.player_pos[:]
        osim.opponent_health    = es.player_health
        osim.opponent_orientation = es.player_orientation
        osim.opponent_gun       = es.player_gun

    # Episode 統計
    print(f"\n=== Episode {ep} Summary ===")
    print(f"Player  Reward: {p_total:.2f}")
    print(f"  Hits: {player_agent.hit_count}, Misses: {player_agent.miss_count}, Kills: {player_agent.kill_count}")
    print(f"  Moves: {player_agent.move_count}, TurnL: {player_agent.turn_left_count}, TurnR: {player_agent.turn_right_count}, Switch: {player_agent.switch_count}")
    print(f"Opponent Reward: {o_total:.2f}")
    print(f"  Hits: {opponent_agent.hit_count}, Misses: {opponent_agent.miss_count}, Kills: {opponent_agent.kill_count}")
    print(f"  Moves: {opponent_agent.move_count}, TurnL: {opponent_agent.turn_left_count}, TurnR: {opponent_agent.turn_right_count}, Switch: {opponent_agent.switch_count}")
    print("===================================\n")

    # 決定勝者並存檔權重
    if p_done:
        winner = opponent_agent
        wname = 'opponent'
    elif o_done:
        winner = player_agent
        wname = 'player'
    else:
        if p_total >= o_total:
            winner = player_agent
            wname = 'player'
        else:
            winner = opponent_agent
            wname = 'opponent'
    winner.save_model(f"{wname}_winner_ep{ep}.pth")
    # 載入勝者權重給雙方
    state_dict = winner.q_network.state_dict()
    player_agent.q_network.load_state_dict(state_dict)
    opponent_agent.q_network.load_state_dict(state_dict)
    player_agent.update_target()
    opponent_agent.update_target()

    reset()



=== Episode 0 Summary ===
Player  Reward: -5665.50
  Hits: 10, Misses: 543, Kills: 6
  Moves: 1511, TurnL: 138, TurnR: 221, Switch: 77
Opponent Reward: -9365.90
  Hits: 1, Misses: 21, Kills: 0
  Moves: 1213, TurnL: 91, TurnR: 99, Switch: 1075


=== Episode 1 Summary ===
Player  Reward: -13146.60
  Hits: 0, Misses: 718, Kills: 0
  Moves: 1077, TurnL: 56, TurnR: 304, Switch: 345
Opponent Reward: -12337.60
  Hits: 0, Misses: 28, Kills: 0
  Moves: 2130, TurnL: 89, TurnR: 51, Switch: 202


=== Episode 2 Summary ===
Player  Reward: -12474.50
  Hits: 0, Misses: 284, Kills: 0
  Moves: 1473, TurnL: 258, TurnR: 284, Switch: 201
Opponent Reward: -12890.30
  Hits: 0, Misses: 1, Kills: 0
  Moves: 2028, TurnL: 11, TurnR: 2, Switch: 458


=== Episode 3 Summary ===
Player  Reward: -12616.90
  Hits: 0, Misses: 310, Kills: 0
  Moves: 1579, TurnL: 254, TurnR: 140, Switch: 217
Opponent Reward: -12858.70
  Hits: 0, Misses: 3, Kills: 0
  Moves: 2493, TurnL: 4, TurnR: 0, Switch: 0


=== Episode 4 Summary ==